## --- Day 3: Mull It Over ---
"Our computers are having issues, so I have no idea if we have any Chief Historians in stock! You're welcome to check the warehouse, though," says the mildly flustered shopkeeper at the North Pole Toboggan Rental Shop. The Historians head out to take a look.

The shopkeeper turns to you. "Any chance you can see why our computers are having issues again?"

The computer appears to be trying to run a program, but its memory (your puzzle input) is corrupted. All of the instructions have been jumbled up!

It seems like the goal of the program is just to multiply some numbers. It does that with instructions like mul(X,Y), where X and Y are each 1-3 digit numbers. For instance, mul(44,46) multiplies 44 by 46 to get a result of 2024. Similarly, mul(123,4) would multiply 123 by 4.

However, because the program's memory has been corrupted, there are also many invalid characters that should be ignored, even if they look like part of a mul instruction. Sequences like mul(4*, mul(6,9!, ?(12,34), or mul ( 2 , 4 ) do nothing.

For example, consider the following section of corrupted memory:
```
xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))
```
Only the four highlighted sections are real mul instructions. Adding up the result of each instruction produces 161 (2*4 + 5*5 + 11*8 + 8*5).

Scan the corrupted memory for uncorrupted mul instructions. What do you get if you add up all of the results of the multiplications?

**AI Prompt**

help me write a regex that looks in a line for expression that matches mul(\d+, \d+)

**AI Response**

Let's break this down into steps. Would you like to write the regex together? 

We need to match:
1. The literal text "mul("
2. 1-3 digits
3. A comma and optional space
4. Another 1-3 digits
5. A closing parenthesis

Would you like me to help you construct each part, or would you prefer me to show you the complete regex pattern?

**AI Prompt**

just show me the complete pattern

**AI Response**

Here's the regex pattern you need:

```
mul\((\d{1,3}),\s*(\d{1,3})\)
```

This will match expressions like `mul(44,46)` or `mul(123, 4)`. The parentheses around the digit groups will capture the numbers for easy extraction. Would you like me to explain any part of this pattern?

**AI Prompt**

ok now whats the full expression given a line

**AI Response**

To find all matches in a line using Python, you'll want to use `re.findall()`. Here's the basic structure:

```python
import re
pattern = r'mul\((\d{1,3}),\s*(\d{1,3})\)'
matches = re.findall(pattern, line)
```

Would you like me to explain what this will return and how to process the matches?

In [ ]:
import re
pattern = r'mul\((\d{1,3}),\s*(\d{1,3})\)'
sample_line = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
matches = re.findall(pattern, sample_line)

In [ ]:
matches

[('2', '4'), ('5', '5'), ('11', '8'), ('8', '5')]

In [ ]:
sum(int(m[0])*int(m[1]) for m in matches)

161

In [ ]:
from aocd import get_data
aoc_input = get_data(day=3, year=2024)

In [ ]:
matches = re.findall(pattern, aoc_input)
sum(int(m[0])*int(m[1]) for m in matches)

188741603

## --- Part Two ---
As you scan through the corrupted memory, you notice that some of the conditional statements are also still intact. If you handle some of the uncorrupted conditional statements in the program, you might be able to get an even more accurate result.

There are two new instructions you'll need to handle:

The do() instruction enables future mul instructions.
The don't() instruction disables future mul instructions.
Only the most recent do() or don't() instruction applies. At the beginning of the program, mul instructions are enabled.

For example:
```
xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))
```
This corrupted memory is similar to the example from before, but this time the mul(5,5) and mul(11,8) instructions are disabled because there is a don't() instruction before them. The other mul instructions function normally, including the one at the end that gets re-enabled by a do() instruction.

This time, the sum of the results is 48 (2*4 + 8*5).

Handle the new instructions; what do you get if you add up all of the results of just the enabled multiplications?

**AI Prompt**

plan of attack
we do the groups as before. then we look for the first do or don't using a lookahead 
does that sound reasonable ?

**AI Response**

Yes, that's a reasonable approach! Let's break it down:

1. First get all mul() matches as before with our existing regex
2. Find all do() and don't() positions in the string using a new regex
3. For each mul() match, we'll need to check the most recent do/don't before its position to determine if it should be counted

Would you like help constructing the regex for finding the do/don't positions? Or would you prefer to try that part yourself first?

**AI Prompt**

yeah help me create the regex to find the do/don't positions

**AI Response**

Let's construct the regex for finding do() and don't() positions:

```python
r'do(?:n\'t)?\(\)'
```

This pattern breaks down as:
- `do` - matches literal "do"
- `(?:n't)?` - optionally matches "n't" (the ?: makes it a non-capturing group)
- `\(\)` - matches the literal "()"

When you use this with re.finditer(), it will give you both the matches and their positions. Would you like me to show you how to use it with finditer()?

**AI Prompt**

yes please

**AI Response**

Here's how to use re.finditer() to get the positions and matches:

```python
control_pattern = r'do(?:n\'t)?\(\)'
controls = list(re.finditer(control_pattern, line))

# Each match object in controls will have:
# match.group() - the full matched text ('do()' or 'don't()')
# match.start() - starting position
# match.end() - ending position
```

Would you like to try this with the example line to see what it returns?

In [ ]:
control_pattern = r'do(?:n\'t)?\(\)'
controls = list(re.finditer(control_pattern, sample_line))

In [ ]:
sample_line

'xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))'

**AI Prompt**

hmm why do i get nothing ?

**AI Response**

Ah, looking at the sample line again, I notice the format is slightly different. In the example, it shows "don't()" but in the sample line it appears as "do_not". Let's modify our regex to handle both possible formats:

```python
control_pattern = r'do(?:n\'t|_not)?\(\)'
```

Would you like to try this updated pattern and see if it matches now?

In [ ]:
control_pattern = r'do(?:n\'t|_not)?\(\)'
controls = list(re.finditer(control_pattern, sample_line))
controls

[]

In [ ]:
sample_line

'xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))'

**AI Prompt**

we are still not getting it

**AI Response**

Looking at the sample line more carefully, I notice I made a mistake. The "do_not" in the sample appears to be part of "do_not_mul" - it's not actually a control instruction. Let me check the problem description again...

I see the issue. Let's look at the second example from Part Two which shows the actual format:
```
xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))
```

So we need to match "don't()" and "do()". Let's try this pattern:

```python
control_pattern = r"(?:don't|do)\(\)"
```

Would you like to try this pattern with the second example to see if it works?

In [ ]:
control_pattern = r"do\(\)"
controls = list(re.finditer(control_pattern, sample_line))
controls

[]

In [ ]:
sample_line

'xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))'

In [ ]:
sample_line2= "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

In [ ]:
controls = list(re.finditer(control_pattern, sample_line2))
controls

[<re.Match object; span=(59, 63), match='do()'>]

In [ ]:
controls

[<re.Match object; span=(59, 63), match='do()'>]

In [ ]:
control_pattern = r'do(?:n\'t|_not)?\(\)'
controls = list(re.finditer(control_pattern, sample_line2))
controls

[<re.Match object; span=(20, 27), match="don't()">,
 <re.Match object; span=(59, 63), match='do()'>]

In [ ]:
pattern = r'mul\((\d{1,3}),\s*(\d{1,3})\)'
control_pattern = r'do(?:n\'t|_not)?\(\)'
matches = list(re.finditer(pattern, sample_line2))
controls = list(re.finditer(control_pattern, sample_line2))
controls

[<re.Match object; span=(20, 27), match="don't()">,
 <re.Match object; span=(59, 63), match='do()'>]

In [ ]:
matches

[<re.Match object; span=(1, 9), match='mul(2,4)'>,
 <re.Match object; span=(28, 36), match='mul(5,5)'>,
 <re.Match object; span=(48, 57), match='mul(11,8)'>,
 <re.Match object; span=(64, 72), match='mul(8,5)'>]

In [ ]:
for m in matches:
    print(m.group(), m.end())

mul(2,4) 9
mul(5,5) 36
mul(11,8) 57
mul(8,5) 72


In [ ]:
what if we wanted to find pattern or control_pattern in the same finditer expression

  Cell In[1], line 1
    what if we wanted to find pattern or control_pattern in the same finditer expression
    ^
SyntaxError: expected 'else' after 'if' expression



**AI Prompt**



**AI Response**

To find both patterns in a single finditer expression, we can combine them with the | (OR) operator. Here's how we could modify the regex:

```python
combined_pattern = f'({pattern})|({control_pattern})'
```

This will match either a mul instruction or a control instruction. Would you like to try this and see how it works?

In [ ]:
combined_pattern = f'({pattern})|({control_pattern})'
matches = list(re.finditer(combined_pattern, sample_line2))
matches

[<re.Match object; span=(1, 9), match='mul(2,4)'>,
 <re.Match object; span=(20, 27), match="don't()">,
 <re.Match object; span=(28, 36), match='mul(5,5)'>,
 <re.Match object; span=(48, 57), match='mul(11,8)'>,
 <re.Match object; span=(59, 63), match='do()'>,
 <re.Match object; span=(64, 72), match='mul(8,5)'>]

In [ ]:
result = 0 
ignore = False 
for m in matches:
    exp = m.group()
    start_pos = m.start()
    end_pos = m.end()
    if exp.find('mul')>-1:
        if not ignore: 
            t1, t2 =exp.replace("mul(", "").replace(")", "").split(",")
            result+=int(t1)*int(t2)
            print("adding:", exp)
        else:
            continue
    elif exp.find("don't")>-1:
        ignore = True
    elif exp.find("do()")>-1:
        ignore = False

adding: mul(2,4)
adding: mul(8,5)


In [ ]:
result

48

In [ ]:
combined_pattern = f'({pattern})|({control_pattern})'
matches = list(re.finditer(combined_pattern, sample_line2))
result = 0 
ignore = False 
for m in matches:
    exp = m.group()
    start_pos = m.start()
    end_pos = m.end()
    if exp.find('mul')>-1:
        if not ignore: 
            t1, t2 =exp.replace("mul(", "").replace(")", "").split(",")
            result+=int(t1)*int(t2)
            print("adding:", exp)
        else:
            continue
    elif exp.find("don't")>-1:
        ignore = True
    elif exp.find("do()")>-1:
        ignore = False

print(result)

adding: mul(2,4)
adding: mul(8,5)
48


In [ ]:
combined_pattern = f'({pattern})|({control_pattern})'
matches = list(re.finditer(combined_pattern, aoc_input))
result = 0 
ignore = False 
for m in matches:
    exp = m.group()
    start_pos = m.start()
    end_pos = m.end()
    if exp.find('mul')>-1:
        if not ignore: 
            t1, t2 =exp.replace("mul(", "").replace(")", "").split(",")
            result+=int(t1)*int(t2)
            print("adding:", exp)
        else:
            continue
    elif exp.find("don't")>-1:
        ignore = True
    elif exp.find("do()")>-1:
        ignore = False

print(result)

adding: mul(666,399)
adding: mul(354,686)
adding: mul(449,25)
adding: mul(298,550)
adding: mul(39,588)
adding: mul(255,532)
adding: mul(501,122)
adding: mul(792,556)
adding: mul(459,837)
adding: mul(390,842)
adding: mul(281,40)
adding: mul(19,457)
adding: mul(699,994)
adding: mul(693,719)
adding: mul(561,466)
adding: mul(355,786)
adding: mul(669,354)
adding: mul(691,921)
adding: mul(302,652)
adding: mul(827,262)
adding: mul(636,791)
adding: mul(267,79)
adding: mul(153,490)
adding: mul(429,93)
adding: mul(481,952)
adding: mul(498,420)
adding: mul(40,723)
adding: mul(172,376)
adding: mul(36,67)
adding: mul(553,221)
adding: mul(768,886)
adding: mul(429,845)
adding: mul(130,55)
adding: mul(550,213)
adding: mul(248,215)
adding: mul(9,131)
adding: mul(511,894)
adding: mul(845,706)
adding: mul(968,989)
adding: mul(644,209)
adding: mul(351,869)
adding: mul(449,265)
adding: mul(535,567)
adding: mul(413,321)
adding: mul(647,475)
adding: mul(594,791)
adding: mul(713,498)
adding: mul(19,524)
addin